<a href="https://colab.research.google.com/github/Akanksha1212/BERT-Classification/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup 

In [8]:
import pandas as pd
import numpy as np
import io

Import csv 

In [9]:
from google.colab import files
uploaded = files.upload()

Saving DF1.csv to DF1 (2).csv


In [10]:
data = pd.read_csv(io.StringIO(uploaded['DF1.csv'].decode('utf-8')))

In [11]:
data.head()

,Title,Category,Post Text
0,Testing requirements in toys and games category,Amazon Custom,My product is domino sets for adults. I am the...
1,Food and Drug Administration,Amazon Custom,"Hello, I am on my way to becoming an Amazon de..."
2,Images Storage for mass upload,Amazon Custom,When uploading products with the spreadsheet w...
3,Prepaid Return Label program email clarification,Amazon Custom,You may have received an email from Amazon inf...
4,Pulling Amazon Custom Personalizatoin Data fro...,Amazon Custom,The problem: Currently Amazon gives you links ...


Make subdata and encode Category 

In [13]:
subdata = data[["Title", "Category", "Post Text"]]
subdata.head()

,Title,Category,Post Text
0,Testing requirements in toys and games category,Amazon Custom,My product is domino sets for adults. I am the...
1,Food and Drug Administration,Amazon Custom,"Hello, I am on my way to becoming an Amazon de..."
2,Images Storage for mass upload,Amazon Custom,When uploading products with the spreadsheet w...
3,Prepaid Return Label program email clarification,Amazon Custom,You may have received an email from Amazon inf...
4,Pulling Amazon Custom Personalizatoin Data fro...,Amazon Custom,The problem: Currently Amazon gives you links ...


In [14]:
subdata['Category'] = subdata['Category'].astype('category')
subdata['Label'] = subdata['Category'].cat.codes.astype('int32')
subdata.sample(5)

,Title,Category,Post Text,Label
10,Total BS! Listing removed for being used and s...,Amazon Custom,"I’m the owner, manufacturer and sole distribut...",1
1811,Need help with account suspension Appeal,Account Health,"Hello members,\nI am requesting for some advic...",0
1246,Deactivated Best Seller but cant appeal (yet),Account Health,I just received a seller performance notice th...,0
1192,POA - Constructive Criticism Needed,Account Health,I had a listing that was removed because of a ...,0
590,Suspen,Fulfillment By Amazon,"Dear Seller,We are writing to remind you that ...",2


Transformer installation from HuggingFace (reload kernel after running for the first time) 

In [15]:
pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 15.2MB/s 
     |████████████████████████████████| 890kB 15.2MB/s 
     |████████████████████████████████| 3.0MB 22.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e3c56efeabe4acceef7e0b71ad684290659481787b269e840c94946ce85729a6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [17]:
# More setup  
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from transformers import DistilBertTokenizer

#### DistilBERT 

In [18]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# For BERT: 
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#### Tokenize 

In [19]:
tokenized = subdata["Title"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [20]:
tokenized

0       [101, 5604, 5918, 1999, 10899, 1998, 2399, 469...
1                      [101, 2833, 1998, 4319, 3447, 102]
2         [101, 4871, 5527, 2005, 3742, 2039, 11066, 102]
3       [101, 17463, 14326, 2709, 3830, 2565, 10373, 1...
4       [101, 4815, 9733, 7661, 3167, 21335, 3406, 237...
                              ...                        
1810    [101, 1037, 6731, 4070, 2029, 2322, 1003, 2013...
1811       [101, 2342, 2393, 2007, 4070, 8636, 5574, 102]
1812                  [101, 7775, 4031, 3343, 13302, 102]
1813    [101, 1022, 2243, 2006, 2907, 1011, 14768, 612...
1814                 [101, 4070, 26709, 6593, 21967, 102]
Name: Title, Length: 1815, dtype: object

#### Padding 

In [21]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i) # Find longest 

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) # Adding 0's 

In [22]:
np.array(padded).shape

(1815, 46)

#### Masking 

In [23]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1815, 46)

#### Run through BERT 

In [24]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [26]:
last_hidden_states

(tensor([[[-0.1835, -0.0325, -0.0348,  ..., -0.3988,  0.1470,  0.3601],
          [ 0.4766,  0.0457, -0.1548,  ..., -0.4201,  0.3834, -0.1362],
          [ 0.1143,  0.2136, -0.0381,  ..., -0.4316, -0.3119,  0.0553],
          ...,
          [ 0.0537, -0.0326,  0.1003,  ..., -0.1306,  0.0262,  0.1715],
          [ 0.1237,  0.0145,  0.3692,  ..., -0.2551,  0.1161,  0.0741],
          [ 0.2589,  0.0167,  0.1903,  ..., -0.2201, -0.0424,  0.0101]],
 
         [[-0.2713, -0.0437, -0.6490,  ..., -0.2204,  0.2402,  0.4417],
          [-0.0381,  0.9230, -0.4835,  ..., -0.2055,  0.3211, -0.7571],
          [-0.3356,  0.3825, -0.5847,  ...,  0.3219,  0.2007,  0.2282],
          ...,
          [ 0.2153, -0.1145, -0.0715,  ...,  0.1220, -0.2260,  0.2741],
          [ 0.2495, -0.1307, -0.0541,  ...,  0.0763, -0.2571,  0.3050],
          [ 0.2494, -0.0476, -0.1152,  ...,  0.0678, -0.2254,  0.2886]],
 
         [[-0.3204, -0.2543, -0.0767,  ..., -0.2676, -0.1091,  0.2475],
          [ 0.1039,  0.0143,

#### Classification 

Get features  

In [27]:
features = last_hidden_states[0][:,0,:].numpy()

In [28]:
labels = subdata["Label"] 

Train Test Split 

In [29]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

Logistic Regression (Can try Decsion Tree, Random Forest, etc.) 

In [30]:
log = LogisticRegression()
log.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 

In [31]:
log.score(test_features, test_labels)

0.751101321585903